In [ ]:
from qch2025.pkg.models.LSTM import network as network
from qch2025.pkg.models.LSTM import func as func

from qch2025.pkg.dataset import DS

import qch2025.pkg.plotting as plt

import numpy as np
import pandas as pd
import torch
import datetime

n_epochs = 35
window_size = 128
batch_size = 32
window_steps = 4 # Minimize the overlap
attn_heads = 4

learning_rate = 0.0001
dropout = 0.1

r2_priority = 0.1

dtype = torch.float32
device = torch.device("cuda")

# Training
train_dataset = DS(dataset_path="/home/ubuntu/repos/quantchallenge-2025/qch2025/dataset/train.csv",
                   window_size=window_size,
                   window_steps=window_steps,
                   dtype=dtype)
evaluate_dataset = DS(dataset_path="/home/ubuntu/repos/quantchallenge-2025/qch2025/dataset/train.csv",
                   window_size=window_size,
                   window_steps=window_steps,
                   dtype=dtype)

rnn = network.RNN(input_size=58, 
                  hidden_size=1024, 
                  lstm_layers=3, 
                  lstm_dropout=dropout, 
                  attn_heads=attn_heads, 
                  output_size=2, 
                  device=device, 
                  dtype=dtype)

losses = func.train(model=rnn, 
                    dataset=train_dataset, 
                    epochs=n_epochs, 
                    learning_rate=learning_rate, 
                    decay=r2_priority)

#plt.plot_line(np.arange(1, len(losses)+1), losses)


#checkpoint = torch.load("/home/ubuntu/repos/quantchallenge-2025/weights/weights.pth", weights_only=True)
#rnn.load_state_dict(checkpoint['model_state_dict'])

eval = DS(dataset_path="/home/ubuntu/repos/quantchallenge-2025/qch2025/dataset/test.csv",
          window_size=window_size,
          window_steps=window_size,
          eval=True,
          device=torch.device("cuda"),
          dtype=dtype) # Create non-overlapping windows
y1, y2 = func.eval(model=rnn, 
                   dataset=eval, 
                   batch_size=batch_size)

df = pd.DataFrame({"Y1": y1, "Y2": y2})
df.index.name="id"
df.index= df.index + 1

actual = len(eval.df["A"])
df1_trunc = df.head(actual)

df1_trunc.to_csv(f"/home/ubuntu/repos/quantchallenge-2025/qch2025/outputs/{datetime.datetime.now()}_predicted.csv")






          A         B         C         D         E         F         G  \
0  0.128342 -0.150355 -0.627005 -0.585210 -0.256904 -0.002869 -0.458575   
1  0.111149 -0.252733  0.042513 -0.556067 -0.127410 -0.232383  0.027353   
2 -0.197761 -0.554981 -0.211682 -0.744126 -0.177530 -0.493903 -0.082152   
3  0.129840 -0.296321  0.509336 -0.567932  0.716634 -0.344869  0.735615   
4  0.022574 -0.342591  0.172240 -0.686743 -0.155533 -0.167701 -0.237098   

          H         I         J  ...  K_neg    L_diff  L_pos  L_neg    M_diff  \
0 -1.425916 -0.378180 -0.661343  ...      1  0.000001      0      1 -0.000002   
1  0.092018 -0.552405 -0.248102  ...      1 -0.235708      0      1  0.428053   
2  0.675504 -0.632501  0.194827  ...      1 -0.304842      0      1  0.251707   
3  1.312159 -0.426990  0.517681  ...      1  0.209939      0      1  0.239458   
4  0.832055 -0.681470 -0.259371  ...      1 -0.073024      0      1 -0.321842   

   M_pos  M_neg    N_diff  N_pos  N_neg  
0      0      1 -0.0

In [ ]:
from qch2025.pkg.models.RNN import network as network
from qch2025.pkg.models.RNN import func as func

from qch2025.pkg.dataset import DS

import qch2025.pkg.plotting as plt

import numpy as np
import pandas as pd
import torch
import datetime

n_epochs = 35
window_size = 128
batch_size = 32
window_steps = 4 # Minimize the overlap
attn_heads = 4

learning_rate = 0.0001
dropout = 0.2

r2_priority = 0.1

dtype = torch.float32

rnn = network.RNN(58, 512, 3, dropout, attn_heads, 2, device=torch.device("cuda"), dtype=dtype)


checkpoint = torch.load("/home/ubuntu/repos/quantchallenge-2025/weights/weights_0.0023788.pth", weights_only=True)
rnn.load_state_dict(checkpoint['model_state_dict'])

eval = DS(dataset_path="/home/ubuntu/repos/quantchallenge-2025/qch2025/dataset/test.csv",
          dataset_path_2="/home/ubuntu/repos/quantchallenge-2025/qch2025/dataset/test_new.csv",
                    window_size=window_size,
                    window_steps=window_size,
                    eval=True,
                    device=torch.device("cuda"),
                    dtype=dtype)
y1, y2 = func.eval(rnn, eval, batch_size=32)
df = pd.DataFrame({"Y1": y1, "Y2": y2})
df.index.name="id"
df.index= df.index + 1

actual = len(eval.df["A"])
df1_trunc = df.head(actual)

df1_trunc.to_csv(f"/home/ubuntu/repos/quantchallenge-2025/qch2025/outputs/{datetime.datetime.now()}_predicted.csv")



Evaluation mode activated, created training items
Out of range by: 4 items!
torch.Size([16000, 2]) 16000 torch.Size([16000, 1]) 2
tensor([[ 0.6544],
        [-0.5561],
        [-0.2648],
        ...,
        [ 0.4971],
        [ 0.2935],
        [ 0.1885]], device='cuda:0')
